Figuring out how to use PAM

In [ ]:
import os
from copy import deepcopy

import acbm

import numpy as np
import pandas as pd
import geopandas as gpd
import pam
from pam import read, write
from pam.activity import Activity, Leg, Plan
from pam.location import Location
from pam.planner.choice_location import DiscretionaryTripOD, DiscretionaryTrips
from pam.planner.od import ODFactory, ODMatrix
from pam.planner.utils_planner import get_trip_chains_either_anchor
from pam.plot.stats import plot_activity_times, plot_leg_times
from pam.utils import minutes_to_datetime as mtdt
from pam.variables import END_OF_DAY
from prettytable import PrettyTable
from shapely.geometry import Point
from libpysal.weights import Queen


from acbm.preprocessing import nts_filter_by_year, add_location
from acbm.assigning.primary_select import select_facility
#from acbm.logger_config import assigning_secondary_locations_logger as logger


pd.set_option('display.max_columns', None)


# Load in the data

In [ ]:
activity_chains = pd.read_parquet(
    acbm.root_path / "data/interim/matching/spc_with_nts_trips.parquet"
)
activity_chains = activity_chains[activity_chains["TravDay"] == 3]  # Wednesday


## Remove all people who don't start their day at home

In [ ]:
activity_chains.head(20)

## Add OA21CD to the data

We will use it to select home locations using select_facility()

In [ ]:
where_clause = "MSOA21NM LIKE '%Leeds%'"

boundaries = gpd.read_file(
    acbm.root_path / "data/external/boundaries/oa_england.geojson", where=where_clause
)

# convert boundaries to 4326
boundaries = boundaries.to_crs(epsg=4326)


# --- Assign activity home locations to boundaries zoning system

# Convert location column in activity_chains to spatial column
centroid_layer = pd.read_csv(
    acbm.root_path / "data/external/centroids/Output_Areas_Dec_2011_PWC_2022.csv"
)
activity_chains = add_location(
    activity_chains, "EPSG:27700", "EPSG:4326", centroid_layer, "OA11CD", "OA11CD"
)

# Convert the DataFrame into a GeoDataFrame, and assign a coordinate reference system (CRS)
activity_chains = gpd.GeoDataFrame(activity_chains, geometry="location")
activity_chains.crs = "EPSG:4326"  # I assume this is the crs


# remove index_right column from activity_chains if it exists
if "index_right" in activity_chains.columns:
    activity_chains = activity_chains.drop(columns="index_right")


# Spatial join to identify which polygons each point is in
activity_chains = gpd.sjoin(
    activity_chains, boundaries[["OA21CD", "geometry"]], how="left", predicate="within"
)
activity_chains = activity_chains.drop("index_right", axis=1)

In [ ]:
# remove location column
activity_chains = activity_chains.drop(columns="location")

## Primary locations

In [ ]:
activity_chains_edu = pd.read_pickle(
    acbm.root_path / "data/interim/assigning/activity_chains_education.pkl"
)

activity_chains_work = pd.read_pickle(
    acbm.root_path / "data/interim/assigning/activity_chains_work.pkl"
)


In [ ]:
activity_chains_edu.head(10)

In [ ]:
# get all activity chains where dact is home
activity_chains_home = activity_chains[activity_chains["dact"] == "home"]
# get all activity chains where dact is not work or education
activity_chains_other = activity_chains[
    ~activity_chains["dact"].isin(["work", "education", "home"])
]


In [ ]:
# Replace ozone and dzone with Na in activity_chains_other. They are incorrect and will be populated later
activity_chains_other.loc[:, ["ozone", "dzone"]] = np.nan
activity_chains_other.head(10)

In [ ]:
(activity_chains.shape[0], 
 activity_chains_edu.shape[0], 
 activity_chains_work.shape[0], 
 activity_chains_home.shape[0], 
 activity_chains_other.shape[0])

In [ ]:
activity_chains["dact"].value_counts()

# Add home locations


In [ ]:
# osm data
osm_data_gdf = gpd.read_parquet(
    acbm.root_path / "data/external/boundaries/west-yorkshire_epsg_4326.parquet"
)

# get rows in osm_data_gdf where activities includes home

osm_data_gdf = osm_data_gdf[osm_data_gdf["activities"].str.contains("home")]
osm_data_gdf

# spatial join to identify which zone each point in osm_data is in
osm_data_gdf = gpd.sjoin(
    osm_data_gdf, boundaries[["OA21CD", "geometry"]], how="inner", predicate="within"
)

osm_data_gdf.head(10)

## Calculate a home location only once per household

In [ ]:
# Keep one row per household and select only household and OA21CD columns
activity_chains_home_hh = activity_chains_home.drop_duplicates(subset=["household"])
activity_chains_home_hh = activity_chains_home_hh[["household", "dact", "OA21CD"]]

In [ ]:
activity_chains_home_hh

## Get the home location

In [ ]:
zone_neighbors = Queen.from_dataframe(boundaries, ids="OA21CD").neighbors


In [ ]:
# apply the function to a row in activity_chains_ex
# TODO: edit function so that we have replacement = false option. We don't want to assign different households to the same home
activity_chains_home_hh[["activity_id", "activity_geom"]] = activity_chains_home_hh.apply(
    lambda row: select_facility(
        row=row,
        facilities_gdf=osm_data_gdf,
        row_destination_zone_col="OA21CD",
        row_activity_type_col="dact",
        gdf_facility_zone_col="OA21CD",
        gdf_facility_type_col="activities",
        gdf_sample_col="floor_area",
        neighboring_zones=zone_neighbors,
    ),
    axis=1,
)

In [ ]:
activity_chains_home_hh.head(5)

## Merge home locations back onto activity_chains_home

In [ ]:
# join activity_chains_home_hh onto activity_chains_home based on household column
activity_chains_home = activity_chains_home.merge(
    activity_chains_home_hh[["household", "activity_id", "activity_geom"]],
    on="household",
    how="left",
)

activity_chains_home.head(5)


In [ ]:
# replace dzone column with OA21CD
activity_chains_home["dzone"] = activity_chains_home["OA21CD"]
activity_chains_home.head(10)

# Combine all dataframes

In [ ]:
# merge the three dataframes
activity_chains_all = pd.concat([activity_chains_edu, 
                                 activity_chains_work, 
                                 activity_chains_home,
                                 activity_chains_other])
# sort by houshold_id, individual_id, and sequence
activity_chains_all = activity_chains_all.sort_values(by=["household", "id", "seq"])
activity_chains_all.head(10)

## Remove all people who don't start their day at home

They will raise an error in PAM

In [ ]:
# group by id column, and remove all groups where oact is not home in the first row
activity_chains_all = activity_chains_all.sort_values(by=["household", "id", "seq"])

print(f'PRE-FILTERING: Number of activities: {activity_chains_all.shape[0]}, number of individuals: {activity_chains_all["id"].nunique()}')
total_activities = activity_chains_all.shape[0]

activity_chains_all = activity_chains_all.groupby("id").filter(lambda x: x.iloc[0]["oact"] == "home")

print(f'POST-FILTERING: Number of activities: {activity_chains_all.shape[0]}, number of individuals: {activity_chains_all["id"].nunique()}')

removed_activities = total_activities - activity_chains_all.shape[0]
percentage_removed = (removed_activities / total_activities) * 100
print(f'Removed {removed_activities} activities, which is {percentage_removed:.2f}% of the total activities')

## Check modes

We can only use modes that we have travel times for

In [ ]:
activity_chains_all["mode"].value_counts()

In [ ]:
# replace motorcyle with car
activity_chains_all["mode"] = activity_chains_all["mode"].replace("motorcycle", "car")

## Populate ozone column for primary activities

Our dfs have populated the `dzone`, `activity_id`, and `activity_geom` columns for rows where `dact` matches: [home, work, education]. 
For each person, we look at rows where the `ozone` is one of [home, work, education], and populate the `ozone`, `origin_id`, `origin_geom` columns for the primary activity with the same value.

TODO: rename `activity_id` to `destination_id` and `activity_geom` to `destination_geom`

In [ ]:
# Step 1: Create dictionaries to map each id to their activity_geom, activity_id, and dzone for each activity type
activity_types = ["home", "education", "work"]
activity_geom_dict = {}
activity_id_dict = {}
dzone_dict = {}

for activity in activity_types:
    filtered_df = activity_chains_all[
        (activity_chains_all["dact"] == activity) & (activity_chains_all["activity_geom"].notnull())
    ]
    activity_geom_dict[activity] = filtered_df.set_index("id")["activity_geom"].to_dict()
    activity_id_dict[activity] = filtered_df.set_index("id")["activity_id"].to_dict()
    dzone_dict[activity] = filtered_df.set_index("id")["dzone"].to_dict()

# Step 2: Populate the origin_geom, origin_id, and ozone columns based on the oact value
def get_origin_geom(row):
    if row["oact"] in activity_geom_dict and row["id"] in activity_geom_dict[row["oact"]]:
        return activity_geom_dict[row["oact"]][row["id"]]
    return None

def get_origin_id(row):
    if row["oact"] in activity_id_dict and row["id"] in activity_id_dict[row["oact"]]:
        return activity_id_dict[row["oact"]][row["id"]]
    return None

def get_ozone(row):
    if row["oact"] in dzone_dict and row["id"] in dzone_dict[row["oact"]]:
        return dzone_dict[row["oact"]][row["id"]]
    return np.nan

activity_chains_all = activity_chains_all.copy()
activity_chains_all["origin_geom"] = activity_chains_all.apply(get_origin_geom, axis=1)
activity_chains_all["origin_id"] = activity_chains_all.apply(get_origin_id, axis=1)
activity_chains_all["ozone"] = activity_chains_all.apply(get_ozone, axis=1)

In [ ]:
activity_chains_all = activity_chains_all[["id", "household", "nts_ind_id", "nts_hh_id", "age_years", 
                                           "oact", "dact", "TripTotalTime", "TripDisIncSW", "seq", "mode", "tst", "tet", 
                                           "ozone", "dzone", "origin_id", "origin_geom", "activity_id", "activity_geom"]]

activity_chains_all.head(10)

### PAM needs a hzone column. Add it

In [ ]:

# Merge the DataFrames on 'household_id' from activity_chains and 'house_id' from activity_chains_home_hh
activity_chains_all = activity_chains_all.merge(activity_chains_home_hh[['household', 'OA21CD']], on='household', how ='left')

# Rename the 'OA21CD' column to 'hzone'
activity_chains_all.rename(columns={'OA21CD': 'hzone'}, inplace=True)

activity_chains_all.head(5)

### Add home locations for all rows where oact = "home"

I was populating home data based on its existence in the dact column. Some entries are outliers which were missed by this logic.

See :

- activity_chains_all[activity_chains_all["id"] == 808]
- activity_chains_all[activity_chains_all["id"] == 1994]

This adds home data for oact = home (MESSY: REDO)

In [ ]:
# activity_chains_all[activity_chains_all["id"] == 808]
# activity_chains_all[activity_chains_all["id"] == 1994]


In [ ]:
# Rename columns then select necessary ones 
activity_chains_home_hh_selected = activity_chains_home_hh.rename(columns={
    'OA21CD': 'ozone', 
    'activity_id': 'origin_id', 
    'activity_geom': 'origin_geom'
})[['household', 'ozone', 'origin_id', 'origin_geom']]

# Merge activity_chains_all with activity_chains_home_hh_selected on 'household'
merged_df = activity_chains_all.merge(activity_chains_home_hh_selected, on='household', how='left', suffixes=('', '_new'))

# Update only the rows where 'oact' is 'home'
home_mask = merged_df['oact'] == 'home'
for column in ['ozone', 'origin_id', 'origin_geom']:
    merged_df.loc[home_mask, column] = merged_df.loc[home_mask, f'{column}_new']

# Drop the temporary columns
merged_df.drop(columns=[f'{column}_new' for column in ['ozone', 'origin_id', 'origin_geom']], inplace=True)

# Assign merged_df back to activity_chains_all
activity_chains_all = merged_df

# Print the updated DataFrame
activity_chains_all

# Prepare data for PAM 

## Individuals

In [ ]:
individuals = activity_chains_all[['id', 'household', 'age_years']].drop_duplicates(subset=['id'])

# rename columns
individuals = individuals.rename(columns={"id": "pid", "household": "hid"})
individuals.head(10)

## Households

## Trips 

In [ ]:
trips = activity_chains_all[['id', 'household', 'seq', 'hzone', 'ozone', 'dzone', 'dact', 'mode', 'tst', 'tet']]

# rename columns
trips = trips.rename(columns={"id": "pid", "household": "hid", "dact": "purp"})

# Drop NA values in tst and tet columns and convert to int
trips = trips.dropna(subset=['tst', 'tet'])
trips['tst'] = trips['tst'].astype(int)
trips['tet'] = trips['tet'].astype(int)

trips.head(10)

In [ ]:
# replace Nan values in ozone and dzone with "na"
trips['ozone'] = trips['ozone'].apply(lambda x: None if pd.isna(x) else x)
trips['dzone'] = trips['dzone'].apply(lambda x: None if pd.isna(x) else x)
trips.head(100)

## Read population 

tour_based = False assumes all trips start from home - this is ok for matsim 
see here https://arup-group.github.io/pam/latest/reference/pam/read/diary/#pam.read.diary.load_travel_diary


In [ ]:
population = pam.read.load_travel_diary(
    trips = trips,
    persons_attributes = individuals,
    tour_based = False
    #hhs_attributes = None,
    )



In [ ]:
plot_activity_times(population)

In [ ]:
plot_leg_times(population)

## Matrices for OD Factory 

The PAM OD factory function needs the following matrices

- travel times (by mode)
- travel distances (this appears optional so I will ignore it for now)
- od_probs: probability of travelling between each pair of zones (by mode)

### Get data: Travel time matrices

In [ ]:
travel_times = pd.read_parquet(
    acbm.root_path / "data/external/travel_times/oa/travel_time_matrix_acbm.parquet"
)

travel_times

#### Edit modes

We have travel times for PT by time of day. In discretionary trips, PAM needs the mode column to match the mode labels in ODFactory (see https://github.com/arup-group/pam/blob/main/examples/17_advanced_discretionary_locations.ipynb). We have two options

1. TODO: Preferred: Before reading the population into PAM, edit the mode column of the trips table to replace pt with pt_wkday_morning, pt_wkday_evening etc depending on day and time of trip. I dont know if this will work downstream
2. Simplify our travel time data. Use the same travel time regardless of time of day, and label as pt (to match with mode column)

I will do 2 for now

In [ ]:
# keep only the rows that match specific "combination" values

modes_to_use = ['car', 'walk', 'cycle', 'pt_wkday_morning']

# Filter the DataFrame
travel_times = travel_times[travel_times['combination'].isin(modes_to_use)]

# Rename specific values in "combination" column
travel_times['combination'] = travel_times['combination'].replace({
    'cycle': 'bike',
    'pt_wkday_morning': 'pt'
})

#### Add OA21CD

In [ ]:
# convert from_id and to_id to int to match the boundaries data type
travel_times = travel_times.astype({"from_id": int, "to_id": int})

# merge travel_times with boundaries
travel_times = travel_times.merge(
    boundaries[["OBJECTID", "OA21CD"]],
    left_on="from_id",
    right_on="OBJECTID",
    how="left",
)
travel_times = travel_times.drop(columns="OBJECTID")

travel_times = travel_times.merge(
    boundaries[["OBJECTID", "OA21CD"]],
    left_on="to_id",
    right_on="OBJECTID",
    how="left",
    suffixes=("_from", "_to"),
)
travel_times = travel_times.drop(columns="OBJECTID")

travel_times.head(10)

### Get data: OD probabilities

We use the activities_per_zone data to calculate the OD probabilities

In [ ]:
activities_per_zone = pd.read_parquet(
    acbm.root_path / "data/interim/assigning/activities_per_zone.parquet"
)

activities_per_zone.head(5)

In [ ]:
# keep only rows that don't match primary activities
activities_per_zone = activities_per_zone[activities_per_zone["activity"].isin(["shop", "other", "medical", "visit"])]

# group by zone and get sum of counts and floor_area
activities_per_zone = activities_per_zone.groupby("OA21CD").agg({"counts": "sum", "floor_area": "sum"}).reset_index()
activities_per_zone.head(5)

In [ ]:
# Merge to get floor_area for origin
merged_df = travel_times.merge(activities_per_zone, left_on='OA21CD_to', right_on='OA21CD')

# Calculate the visit_probability: it is a funciton of floor_area and travel time
merged_df['visit_prob'] = np.where(merged_df['travel_time_p50'] != 0, 
                              round(merged_df['floor_area'] / np.sqrt(merged_df['travel_time_p50'])), 
                              round(merged_df['floor_area'])
                              )

merged_df

### Create matrices

In [ ]:
def create_od_matrices(
        df: pd.DataFrame, 
        mode_column: str, 
        value_column: str,
        zone_labels: tuple,
        fill_value: int,
        zone_from: str = 'OA21CD_from',
        zone_to: str = 'OA21CD_to',
        ) -> dict:
    
    """
    Create OD matrices for each mode in the DataFrame. This function is uused to create matrices for 
    - travel times
    - od_probs
    to be used in discretionary activity selection

    Parameters
    ----------
    df : pd.DataFrame
        DataFrame containing the data
    mode_column : str
        Column name containing the mode of transport
    value_column : str
        Column name containing the value to be used in the OD matrix
    fill_value : int
        Value to use when a value for a specific od pair is not available

    Returns
    -------
    dict
        A dictionary containing OD matrices for each mode.
        Key: str
            Mode of transport
        Value: np.array
            OD matrix

    """
    
    # Initialize dictionaries to hold OD matrices for each combination type
    modes = df[mode_column].unique()
    od_matrices = {mode: np.full((len(zone_labels), len(zone_labels)), fill_value) for mode in modes}
    
    # Create a mapping from zone labels to indices
    zone_index = {label: idx for idx, label in enumerate(zone_labels)}
    
    # Vectorized operation to populate OD matrices
    from_indices = df[zone_from].map(zone_index)
    to_indices = df[zone_to].map(zone_index)
    
    for mode in modes:
        print(f"Starting mode: {mode}")
        mask = df[mode_column] == mode
        values = df[mask][value_column].fillna(fill_value)  # Fill missing values 
        od_matrices[mode][from_indices[mask], to_indices[mask]] = values
        print(f"Finished mode: {mode}")
    
    return od_matrices



In [ ]:
# Extract unique zone labels. 
# TODO: get these from boundary/zone layer instead
zone_labels = pd.unique(travel_times[['OA21CD_from', 'OA21CD_to']].values.ravel('K'))
zone_labels = tuple(zone_labels)
print(zone_labels[:5])

#### Create travel time matrices

In [ ]:
matrix_travel_times = create_od_matrices(
    df = merged_df, 
    mode_column = 'combination', 
    value_column = 'travel_time_p50', 
    zone_labels = zone_labels,
    fill_value = 300,  # replace missing travel times with 6 hours (they are unreachable)
    zone_from='OA21CD_from', 
    zone_to='OA21CD_to'
    )

In [ ]:
matrix_travel_times["car"][100:110, 100:110]

#### Create od probs matrices

In [ ]:
matrix_od_probs = create_od_matrices(
    df = merged_df, 
    mode_column = 'combination', 
    value_column = 'visit_prob', 
    zone_labels = zone_labels,
    # replace missing probabilities with 1. There are no activities so shouldn't be visited
    # 1 used instead of 0 to avoid (ValueError: Total of weights must be finite) in weighted sampling 
    # (https://github.com/arup-group/pam/blob/c8bff760fbf92f93f95ff90e4e2af7bbe107c7e3/src/pam/planner/utils_planner.py#L17)
    fill_value = 1,  
    zone_from='OA21CD_from', 
    zone_to='OA21CD_to'
    )

In [ ]:
#matrix_od_probs["walk"][100:110, 100:110]
matrix_od_probs["walk"][0:10, 0:10]


#### Create ODMatrix objects

In [ ]:
mode_types = travel_times['combination'].unique()

In [ ]:
matrices_pam_travel_time = [
    ODMatrix("time", mode, zone_labels, zone_labels, matrix_travel_times[mode])
    for mode in mode_types
]

In [ ]:
matrices_pam_travel_time[1]

In [ ]:
matrices_pam_od_probs = [
    ODMatrix("od_probs", mode, zone_labels, zone_labels, matrix_od_probs[mode])
    for mode in mode_types
]

#### Create ODFactory object 

In [ ]:
# combine ODMatrix objects
matrices_pam_all = matrices_pam_travel_time + matrices_pam_od_probs
matrices_pam_all

In [ ]:
#create ODFactory
od = ODFactory.from_matrices(matrices = matrices_pam_all)


In [ ]:
od

### Discretionary activities

Test the implementation of discretionary activities

In [ ]:
def print_activity_locs(plan):
    summary = PrettyTable(["seq", "purpose", "location"])
    for seq, act in enumerate(plan.activities):
        summary.add_row([seq, act.act, act.location.area])
    print(summary)

In [ ]:
import random

plans_iterator = population.plans()
all_plans = list(plans_iterator)


random_plan = random.choice(all_plans)
print_activity_locs(random_plan)

In [ ]:
random_plan_copy = deepcopy(random_plan)
planner = DiscretionaryTrips(plan=random_plan_copy, od=od)
planner.update_plan()

print_activity_locs(random_plan_copy)

### Apply logic to entire population

In [ ]:
i = 0
people_list = list(population.people())
for plan in population.plans():
    try:
        planner = DiscretionaryTrips(plan=plan, od=od)
        planner.update_plan()
        print(f"Updated plan for person id {people_list[i][1]}")
    except Exception as e:
        # a pam population.people() object has hid, pid, <plan>. We want pid
        print(f"An error occurred with person id {people_list[i][1]}: {e}")
    i += 1



In [ ]:
def update_population_plans(population: pam.core.Population, 
                            od: ODFactory
                            ) -> None:
    """
    Update the plans in a population object using the DiscretionaryTrips planner

    """
    i = 0
    people_list = list(population.people())
    for plan in population.plans():
        try:
            planner = DiscretionaryTrips(plan=plan, od=od)
            planner.update_plan()
            logger.info(f"Updated plan for person id {people_list[i][0]}")
        except Exception as e:
            logger.error(f"An error occurred with person id {people_list[i][0]}: {e}")
        i += 1

update_population_plans(population, od)

## Save

In [ ]:
write.to_csv(population, dir=(
    acbm.root_path / "data/processed/activities_pam"
))

# Read activities back and assign point locaitons

In [ ]:
activities_pam = pd.read_csv(
    acbm.root_path / "data/processed/activities_pam/activities.csv"
)

In [ ]:
# select only the columns we need
activities_pam = activities_pam[['pid', 'hid', 'activity', 'zone']]
# select all rows where activity is not home, work, education
activities_pam = activities_pam[~activities_pam['activity'].isin(['home', 'work', 'education'])]

In [ ]:
activities_pam["activity"].value_counts()

## Select facility

In [ ]:
# apply the function to a row in activity_chains_ex
activities_pam[["activity_id", "activity_geom"]] = activities_pam.apply(
    lambda row: select_facility(
        row=row,
        facilities_gdf=osm_data_gdf,
        row_destination_zone_col="zone",
        row_activity_type_col="activity",
        gdf_facility_zone_col="OA21CD",
        gdf_facility_type_col="activities",
        gdf_sample_col="floor_area",
        fallback_to_random=True,
        neighboring_zones=zone_neighbors,
    ),
    axis=1,
)

## Add locations of secondary activities onto the original activity chains

In [ ]:
activity_chains_all